# evaluation

Evaluate both baseline and TabTransformer models with test set


In [10]:
import keras_preprocessing
from keras import layers
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa

from pathlib import Path
import pandas as pd
import numpy as np
import os


print(tf.config.list_physical_devices("GPU"))


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [11]:
import pandas as pd
import random
import sklearn

from pathlib import Path
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTENC


In [12]:
CSV_HEADER = [
    "gender",
    "age",
    "hypertension",
    "heart_disease",
    "ever_married",
    "work_type",
    "residence_type",
    "avg_glucose_level",
    "bmi",
    "smoking_status",
    "stroke",
]


CATEGORICAL_FEATURE_NAMES = [
    "gender",
    "hypertension",
    "heart_disease",
    "ever_married",
    "work_type",
    "residence_type",
    "smoking_status",
]

NUMERIC_FEATURE_NAMES = ["age", "avg_glucose_level", "bmi"]


def encode_int(data: pd.DataFrame, categorical_features: list[str]):
    return pd.get_dummies(data, columns=categorical_features, drop_first=True)


def split_label(data: pd.DataFrame):
    x = data.copy().drop("stroke", axis=1)
    y = data["stroke"]  # labels

    return x, y


def resample(data: pd.DataFrame, seed: int, categorical_features: list[str]):
    """oversample positive cases with SMOTE and undersample negative with EEN"""
    # encode categorical features first
    enc = OrdinalEncoder()
    data[categorical_features] = enc.fit_transform(data[categorical_features])

    X = data.drop(columns=["stroke"], axis=1)
    Y = data["stroke"]

    cat_features_indices = [
        data.columns.get_loc(label) for label in categorical_features
    ]

    smote_nc = SMOTENC(categorical_features=cat_features_indices, random_state=seed)
    smote_een = SMOTEENN(smote=smote_nc, random_state=seed, sampling_strategy="auto")

    x_resampled, y_resampled = smote_een.fit_resample(X, Y)
    x_resampled["stroke"] = y_resampled

    x_resampled[categorical_features] = enc.inverse_transform(
        x_resampled[categorical_features]
    )

    return pd.DataFrame(x_resampled, columns=data.columns)


def scale(df):
    X_num = df[NUMERIC_FEATURE_NAMES]
    X_cat = df[CATEGORICAL_FEATURE_NAMES]

    scaler = StandardScaler()
    scaler.fit(X_num)

    X_scaled = scaler.transform(X_num)
    X_scaled = pd.DataFrame(X_scaled, index=X_num.index, columns=X_num.columns)

    df_scaled = pd.concat([X_scaled, X_cat, df["stroke"]], axis=1)[df.columns]

    return df_scaled


def split_train_valid_test(data_df, seed: int, resample_training: bool):
    data_df = data_df.sample(frac=1, random_state=seed)

    test_set = data_df[round(len(data_df) * 0.85) :]
    train_validation_data = data_df[: round(len(data_df) * 0.85)].sample(
        frac=1, random_state=seed
    )

    train_set = train_validation_data[: round(len(data_df) * 0.70)]
    validation_set = train_validation_data[round(len(data_df) * 0.70) :]

    if resample_training:
        train_set = resample(
            train_validation_data[: round(len(data_df) * 0.70)],
            seed,
            categorical_features=CATEGORICAL_FEATURE_NAMES,
        )

    return train_set, validation_set, test_set


def prepare_data(seed: int, resample_training: bool):
    data_df = pd.read_csv(Path().resolve().joinpath("dataset/full_data_clean.csv"))

    train_df, validation_df, test_df = split_train_valid_test(
        data_df, seed, resample_training
    )

    train_df, validation_df, test_df = [
        scale(df) for df in [train_df, validation_df, test_df]
    ]

    return train_df, validation_df, test_df


In [13]:
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.0001
DROPOUT_RATE = 0.1
BATCH_SIZE = 32
NUM_EPOCHS = 100

MLP_MODEL_PATH = str(Path().resolve().joinpath("model/mlp_model"))
TABTRANSFORMER_MODEL_PATH = str(Path().resolve().joinpath("model/tabtransformer_model"))

TARGET_FEATURE_NAME = "stroke"
TARGET_LABELS = ["1", "0"]


In [14]:
# data proccessing pipeline

target_label_lookup = layers.StringLookup(
    vocabulary=TARGET_LABELS, mask_token=None, num_oov_indices=0
)


def prepare_example(features, target):
    # target_index = target_label_lookup(target)
    target_index = target
    return features, target_index


def get_dataset_from_csv(csv_file_path, batch_size=128, shuffle=False):
    """dataset from, csv"""
    dataset = tf.data.experimental.make_csv_dataset(
        csv_file_path,
        batch_size=batch_size,
        column_names=CSV_HEADER,
        label_name=TARGET_FEATURE_NAME,
        num_epochs=1,
        header=False,
        na_value="?",
        shuffle=shuffle,
    ).map(prepare_example, num_parallel_calls=tf.data.AUTOTUNE, deterministic=False)
    return dataset.cache()


/home/haoming/.local/lib/python3.10/site-packages/numpy/core/numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [15]:
# training and evaluation


def train(
    model: keras.Model,
    train_data_file,
    test_data_file,
    model_output,
    num_epochs,
    EPOCHS_TO_WAIT_FOR_IMPROVE,
    learning_rate,
    batch_size,
):
    """Implement a training and evaluation procedure"""
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=WEIGHT_DECAY
    )

    train_dataset = get_dataset_from_csv(train_data_file, batch_size, shuffle=True)
    validation_dataset = get_dataset_from_csv(test_data_file, batch_size)

    metrics = (
        [
            keras.metrics.BinaryAccuracy(name="acc"),
            keras.metrics.AUC(name="auc"),
        ],
    )

    early_stop_callback = keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=EPOCHS_TO_WAIT_FOR_IMPROVE
    )
    # checkpoint_callback = keras.callbacks.ModelCheckpoint(model_output, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

    # early_stop_callback = keras.callbacks.EarlyStopping(
    #    monitor="val_auc", patience=EPOCHS_TO_WAIT_FOR_IMPROVE
    # )

    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        model_output,
        monitor="val_auc",
        verbose=1,
        save_best_only=True,
        mode="max",
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics,
    )

    print("Start training the model...")

    history = model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=validation_dataset,
        callbacks=[
            # checkpoint_callback,
            early_stop_callback
        ],
    )

    print("Model training finished")

    _, accuracy, auc = model.evaluate(validation_dataset, verbose=0)

    print(f"Validation accuracy: {round(accuracy * 100, 2)}% AUC: {auc}")

    return history, model


### MLP:

loss: 0.2073 - auc: 0.8241 - accuracy: 0.9397

loss: 0.2145 - auc: 0.8316 - accuracy: 0.9397

### TabTransformer:

loss: 0.2197 - auc: 0.7708 - accuracy: 0.9335


In [16]:
RESULT_COLS = [
    "classifier",
    "precision",
    "recall",
    "fscore",
    "accuracy",
    "auc",
    "miss_rate",
    "fall_out_rate",
]

train_data_path = Path().resolve().joinpath("dataset/train_data.csv")
validation_data_path = Path().resolve().joinpath("dataset/validation_data.csv")
test_data_path = Path().resolve().joinpath("dataset/test_data.csv")

train_data_file = str(train_data_path.absolute())
validation_data_file = str(validation_data_path.absolute())
test_data_file = str(test_data_path.absolute())

NUM_EXPERIMENTS = 10
EPOCHS_TO_WAIT_FOR_IMPROVE = 15


def metrics(y_pred, y_true):
    auc = roc_auc_score(y_true=y_true, y_score=y_pred)

    y_pred = np.where(y_pred > 0.2, 1, 0)

    tn, fp, fn, tp = confusion_matrix(y_pred=y_pred, y_true=y_true).ravel()

    # metrics
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    fscore = 2 * tp / (2 * tp + fp + fn)
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    miss_rate = fn / (tn + tp)
    fall_out_rate = fp / (fp + tn)

    # return
    return [precision, recall, fscore, accuracy, auc, miss_rate, fall_out_rate]


def metrics_keras(model: keras.Model, test_data_file: str):
    model.compile(
        metrics=[
            keras.metrics.AUC(
                num_thresholds=200,
                curve="ROC",
            ),
            keras.metrics.Precision(),
            keras.metrics.Recall(),
            keras.metrics.TrueNegatives(),
            keras.metrics.FalsePositives(),
            keras.metrics.TrueNegatives(),
            keras.metrics.TruePositives(),
        ]
    )

    _, auc, precision, recall, tn, fn, fp, tp = model.evaluate(
        get_dataset_from_csv(test_data_file)
    )

    # metrics
    fscore = 2 * tp / (2 * tp + fp + fn)
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    miss_rate = fn / (tn + tp)
    fall_out_rate = fp / (fp + tn)

    # return
    return [precision, recall, fscore, accuracy, auc, miss_rate, fall_out_rate]


In [17]:
def run_experiment(
    experiment, tabtransformer_model, seed, train_data, validation_data, test_data
):
    # split labels
    train_data_file = str(
        Path().resolve().joinpath(f"dataset/train_data_tt_exp_{experiment}.csv")
    )
    validation_data_file = str(
        Path().resolve().joinpath(f"dataset/validation_data_tt_exp_{experiment}.csv")
    )
    test_data_file = str(
        Path().resolve().joinpath(f"dataset/test_data_tt_exp_{experiment}.csv")
    )

    train_data.to_csv(train_data_file, header=False, index=False)
    validation_data.to_csv(validation_data_file, header=False, index=False)
    test_data.to_csv(test_data_file, header=False, index=False)

    # train tabtransformer model on training data and evaluate on validation data
    history, tt_trained = train(
        model=tabtransformer_model,
        train_data_file=train_data_file,
        test_data_file=validation_data_file,
        model_output=TABTRANSFORMER_MODEL_PATH,
        num_epochs=NUM_EPOCHS,
        EPOCHS_TO_WAIT_FOR_IMPROVE=EPOCHS_TO_WAIT_FOR_IMPROVE,
        learning_rate=LEARNING_RATE,
        batch_size=BATCH_SIZE,
    )

    train_validation_data_file = str(
        Path()
        .resolve()
        .joinpath(f"dataset/train_validation_data_tt_exp_{experiment}.csv")
    )

    pd.concat([train_data, validation_data]).sample(frac=1, random_state=seed).to_csv(
        train_validation_data_file, index=False, header=False
    )

    # now, train tabtransformer model on validation data and evaluate on test data
    history, tt_trained = train(
        model=tt_trained,
        train_data_file=train_validation_data_file,
        test_data_file=test_data_file,
        model_output=TABTRANSFORMER_MODEL_PATH,
        num_epochs=NUM_EPOCHS,
        EPOCHS_TO_WAIT_FOR_IMPROVE=EPOCHS_TO_WAIT_FOR_IMPROVE,
        learning_rate=LEARNING_RATE,
        batch_size=BATCH_SIZE,
    )

    metrics = list(metrics_keras(tt_trained, test_data_file))

    # cleanup
    os.remove(train_data_file)
    os.remove(validation_data_file)
    os.remove(train_validation_data_file)
    os.remove(test_data_file)

    return metrics


In [18]:
tabtransformer_model = keras.models.load_model(TABTRANSFORMER_MODEL_PATH)

results = {}
results_resample = {}

for experiment in range(NUM_EXPERIMENTS):
    seed = random.randint(0, 1000)

    train_data, validation_data, test_data = prepare_data(
        seed=seed, resample_training=False
    )

    results[experiment] = list(
        run_experiment(
            experiment=experiment,
            tabtransformer_model=tabtransformer_model,
            seed=seed,
            train_data=train_data,
            validation_data=validation_data,
            test_data=test_data,
        )
    )

    train_data, validation_data, test_data = prepare_data(
        seed=seed, resample_training=True
    )

    results_resample[experiment] = list(
        run_experiment(
            experiment=experiment,
            tabtransformer_model=tabtransformer_model,
            seed=seed,
            train_data=train_data,
            validation_data=validation_data,
            test_data=test_data,
        )
    )

results = (
    pd.DataFrame()
    .from_dict(results, orient="index", columns=RESULT_COLS)
    .assign(classifier="TabTransformer")
)
results_resample = (
    pd.DataFrame()
    .from_dict(results_resample, orient="index", columns=RESULT_COLS)
    .assign(classifier="TabTransformer")
)

results.to_csv(Path().resolve().joinpath("results/TabTransformer_eval.csv"))
results_resample.to_csv(
    Path().resolve().joinpath("results/TabTransformer_eval_resampled.csv")
)


Start training the model...
Epoch 1/100
950/950 [==============================] - 24s 19ms/step - loss: 0.1020 - acc: 0.9784 - auc: 0.6401 - val_loss: 0.0924 - val_acc: 0.9791 - val_auc: 0.7870
Epoch 2/100
950/950 [==============================] - 16s 17ms/step - loss: 0.0842 - acc: 0.9828 - auc: 0.7059 - val_loss: 0.0907 - val_acc: 0.9791 - val_auc: 0.7968
Epoch 3/100
950/950 [==============================] - 16s 17ms/step - loss: 0.0816 - acc: 0.9829 - auc: 0.7181 - val_loss: 0.0914 - val_acc: 0.9791 - val_auc: 0.7958
Epoch 4/100
950/950 [==============================] - 16s 17ms/step - loss: 0.0803 - acc: 0.9829 - auc: 0.7365 - val_loss: 0.0892 - val_acc: 0.9791 - val_auc: 0.8067
Epoch 5/100
950/950 [==============================] - 16s 17ms/step - loss: 0.0801 - acc: 0.9829 - auc: 0.7375 - val_loss: 0.0912 - val_acc: 0.9791 - val_auc: 0.8014
Epoch 6/100
950/950 [==============================] - 16s 17ms/step - loss: 0.0797 - acc: 0.9829 - auc: 0.7458 - val_loss: 0.0906 - val_

ValueError: No files match `file_pattern` /home/haoming/Projects/python/brain-stroke-prediction/dataset/test_data_tt_exp_0.csv.